In [ ]:
# @title Imports

import os
import mat73
import torch
import shutil
import subprocess
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from utils import init_random_seeds
from sklearn.preprocessing import StandardScaler
from utils import NEURONS_302, ROOT_DIR, VALID_DATASETS
from preprocess._utils import (
    CausalNormalizer,
    interpolate_data,
    pickle_neural_data,
    reshape_calcium_data,
    smooth_data_preprocess,
)

In [ ]:
# @title Setup: Download the dataset and set random seeds

# set random seeds
seed = 42
init_random_seeds(seed)

# download the dataset
dataset_name = "Lin2023"
assert dataset_name in VALID_DATASETS, f"{dataset_name} is not a valid dataset."
config = OmegaConf.load("../configs/submodule/preprocess.yaml")
url = config.preprocess.url
zipfile = config.preprocess.zipfile
zip_path = os.path.join(ROOT_DIR, zipfile)
source_path = os.path.join(ROOT_DIR, zipfile.strip(".zip"))

if not os.path.exists(os.path.join(source_path, dataset_name)):
    download_url(url=url, folder=ROOT_DIR, filename=zipfile)
    # unzip the dataset folder
    bash_command = [
        "unzip",
        zip_path,
        "{}/*".format(dataset_name),
        "-d",
        source_path,
    ]
    std_out = subprocess.run(bash_command, text=True)  # Run the bash command
    print(std_out, end="\n\n")
    # delete the zip file
    os.unlink(zip_path)
else:
    print(f"{dataset_name} dataset already downloaded and unzipped.")

DATA_DIR = os.path.join(source_path, dataset_name)

In [ ]:
# instantiate the relevant preprocessor class
preprocessor = eval(dataset_name + "Preprocessor")(
     StandardScaler(),
     "ES", 
     "linear",
     0.666,
)
# call its method
preprocessor.preprocess()

In [ ]:
pickle_neural_data(
        # Code above does this block
        url=url,
        zipfile=zipfile,
        dataset=dataset_name,
        ################################
        smooth_method="ES",
        resample_dt=0.666,
        interpolate_method="linear",
        cleanup=True,
    )